In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
path = "/content/drive/MyDrive/TFM/"
path1 = path + "Planets.csv"
path2 = path + "Not_Planets.csv"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Creación del dataset con los registros de secuencias positivas y secuencias negativas
planets_df = pd.read_csv(path1, names=list(range(4000)))
not_planets_df = pd.read_csv(path2, names=list(range(4000)))
X_data = pd.concat([planets_df, not_planets_df]).to_numpy()
y_data = np.concatenate((np.ones(len(planets_df)), np.zeros(len(not_planets_df))))

#Separación en train y test
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.20, random_state=42)
y_train = np.transpose(np.atleast_2d(y_train))
y_test = np.transpose(np.atleast_2d(y_test))

In [4]:
#Definición del modelo a entrenar
from tensorflow.keras import layers
from tensorflow import keras

model_name = "Conv1D_A"

model = keras.Sequential(
    [
        layers.Input(shape=(4000,1)),
        layers.Conv1D(filters=64, kernel_size=5, activation='relu'),
        layers.Conv1D(filters=64, kernel_size=5, activation='relu'),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(100, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ],
    name=model_name
)

In [5]:
from datetime import datetime
#Compilación del modelo
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
              keras.metrics.BinaryAccuracy(),
              keras.metrics.Recall(),
              keras.metrics.Precision(),
              keras.metrics.AUC(num_thresholds=25)
            ]
    )

#Definición del directorio donde guardar los datos del entrenamiento
logdir = "/content/drive/MyDrive/TFM/Logs/" + model.name + "/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/TFM/Models/" + model_name + "/best_model.keras",
    verbose=1,
    save_best_only=True,
    mode="max",
    monitor="val_binary_accuracy",
)

In [6]:
training_history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    verbose=1, # Suppress chatty output; use Tensorboard instead
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[tensorboard_callback, model_checkpoint],
)

Epoch 1/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - auc: 0.5473 - binary_accuracy: 0.5464 - loss: 0.6779 - precision: 0.5380 - recall: 0.8025
Epoch 1: val_binary_accuracy improved from -inf to 0.47219, saving model to /content/drive/MyDrive/TFM/Models/Conv1D_A/best_model.keras
520/520 ━━━━━━━━━━━━━━━━━━━━ 46s 74ms/step - auc: 0.5473 - binary_accuracy: 0.5464 - loss: 0.6779 - precision: 0.5380 - recall: 0.8026 - val_auc: 0.5981 - val_binary_accuracy: 0.4722 - val_loss: 0.6871 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
519/520 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - auc: 0.5653 - binary_accuracy: 0.5767 - loss: 0.6640 - precision: 0.5562 - recall: 0.9141
Epoch 2: val_binary_accuracy improved from 0.47219 to 0.58801, saving model to /content/drive/MyDrive/TFM/Models/Conv1D_A/best_model.keras
520/520 ━━━━━━━━━━━━━━━━━━━━ 65s 51ms/step - auc: 0.5653 - binary_accuracy: 0.5767 - loss: 0.6640 - precision: 0.5562 - recall: 0.9141 - val_auc: 0.5682 - val_binary_accuracy: 0.58